In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.extend([ '../', '../../'])
import torch
import torch.nn as nn
import torch.optim as optim
from data import input_data
import dataloader as dl
from args import Args
# from model import Het_En, Classifier, EdgePredictor, Edge_classify
from smote import oversample
from train2 import train_smote
import cProfile

# Set device to GPU if available, else use CPU
args = Args()
args.aminer_edge_pred()
torch.cuda.empty_cache()

Current device: NVIDIA A100 80GB PCIe


In [2]:
# data = input_data(args) 
# torch.save(data, 'data.pt')
data = torch.load('edge_pred_data.pt')
print(data)

HeteroData(
  a={
    num_nodes=20171,
    y=[20171]
  },
  p={ num_nodes=13250 },
  v={ num_nodes=18 },
  p_title_embed={ x=[13250, 128] },
  p_abstract_embed={ x=[13250, 128] },
  p_net_embed={ x=[13250, 128] },
  p_a_net_embed={ x=[13250, 128] },
  p_v_net_embed={ x=[13250, 128] },
  a_net_embed={ x=[20171, 128] },
  a_text_embed={ x=[20171, 128] },
  v_net_embed={ x=[18, 128] },
  v_text_embed={ x=[18, 128] },
  e={
    x=[29166, 2],
    y=[29166]
  },
  (a, walk, a)={ edge_index=[2, 90641] },
  (a, walk, p)={ edge_index=[2, 89734] },
  (a, walk, v)={ edge_index=[2, 28573] },
  (p, walk, a)={ edge_index=[2, 65555] },
  (p, walk, p)={ edge_index=[2, 66190] },
  (p, walk, v)={ edge_index=[2, 19461] },
  (v, walk, a)={ edge_index=[2, 90] },
  (v, walk, p)={ edge_index=[2, 90] },
  (v, walk, v)={ edge_index=[2, 13] }
)


In [3]:
print(data['a','walk','p'].edge_index)
device = args.device
# data = data.to(device)

# Send all x tensors to the device
data['e']['x'] = data['e']['x'].to(device)
data['p_title_embed']['x'] = data['p_title_embed']['x'].to(device)
data['p_abstract_embed']['x'] = data['p_abstract_embed']['x'].to(device)
data['p_net_embed']['x'] = data['p_net_embed']['x'].to(device)
data['p_a_net_embed']['x'] = data['p_a_net_embed']['x'].to(device)
# data['p_p_net_embed']['x'] = data['p_p_net_embed']['x'].to(device)
data['p_v_net_embed']['x'] = data['p_v_net_embed']['x'].to(device)
data['a_net_embed']['x'] = data['a_net_embed']['x'].to(device)
data['a_text_embed']['x'] = data['a_text_embed']['x'].to(device)
data['v_net_embed']['x'] = data['v_net_embed']['x'].to(device)
data['v_text_embed']['x'] = data['v_text_embed']['x'].to(device)

# Send all y tensors to the device
data['a']['y'] = data['a']['y'].to(device)
data['e']['y'] = data['e']['y'].to(device)

# Send all edge_index tensors to the device
data['a', 'walk', 'a']['edge_index'] = data['a', 'walk', 'a']['edge_index'].to(device)
data['a', 'walk', 'p']['edge_index'] = data['a', 'walk', 'p']['edge_index'].to(device)
data['a', 'walk', 'v']['edge_index'] = data['a', 'walk', 'v']['edge_index'].to(device)
data['p', 'walk', 'a']['edge_index'] = data['p', 'walk', 'a']['edge_index'].to(device)
data['p', 'walk', 'p']['edge_index'] = data['p', 'walk', 'p']['edge_index'].to(device)
data['p', 'walk', 'v']['edge_index'] = data['p', 'walk', 'v']['edge_index'].to(device)
data['v', 'walk', 'a']['edge_index'] = data['v', 'walk', 'a']['edge_index'].to(device)
data['v', 'walk', 'p']['edge_index'] = data['v', 'walk', 'p']['edge_index'].to(device)
data['v', 'walk', 'v']['edge_index'] = data['v', 'walk', 'v']['edge_index'].to(device)

edge_indices = [ data['p', 'walk', 'a'].edge_index, data['p', 'walk', 'p'].edge_index, data['p', 'walk', 'v'].edge_index ]
target_edge = data['e']['x']

tensor([[    0,     0,     0,  ..., 20169, 20169, 20170],
        [10523, 10550, 10395,  ...,  9100,  9274,  4378]])


In [4]:
c_train_num = dl.train_num(data['e'].y, args.im_class_num, args.class_samp_num[0], args.im_ratio)
print(c_train_num, sum(c_train_num))
train_idx, val_idx, test_idx, c_num_mat = dl.segregate(data['e'].y, c_train_num, args)
print("train_idx: ", train_idx, "\n", len(train_idx))
print("val_idx: ", val_idx, "\n", len(val_idx))
print("test_idx: ", test_idx, "\n", len(test_idx))
# print(c_num_mat)

[4000, 3600] 7600
0 14583
1 14583
train_idx:  [15339, 11383, 15675, 27255, 23990, 23964, 8607, 12471, 28759, 22826, 9184, 1761, 27991, 24015, 15061, 18006, 14472, 6671, 18810, 7619, 4923, 19470, 17143, 25424, 28871, 17086, 5692, 19075, 1891, 10711, 8640, 10539, 26755, 11496, 27796, 14722, 23601, 9862, 6966, 100, 27338, 10392, 27269, 9598, 23071, 21745, 22027, 10324, 6610, 2748, 18617, 17390, 14197, 28436, 4928, 4110, 24324, 5754, 25351, 27716, 24176, 25973, 19652, 9748, 7902, 5847, 16383, 1501, 25285, 23795, 28676, 16659, 2320, 28690, 13633, 29127, 14453, 13451, 23699, 7912, 7843, 18241, 18249, 12017, 5120, 22445, 18120, 11193, 22341, 15582, 29032, 489, 15053, 22902, 19417, 28650, 13889, 6015, 20076, 12787, 18466, 16002, 3617, 22862, 27367, 6574, 4661, 566, 25662, 14444, 19996, 26401, 2259, 9858, 7148, 14479, 19399, 27321, 13894, 1403, 9296, 17657, 25233, 26709, 22383, 23625, 25589, 24191, 25955, 6932, 1269, 20332, 5168, 1081, 10183, 2542, 6431, 8822, 14489, 3594, 17209, 8124, 11689, 1

In [5]:
# encoder = Het_En(args.embed_dim, args.dropout)
# classifier = Edge_classify(args.embed_dim, args.nclass, args.dropout)
# decoder_m = EdgePredictor(args.embed_dim)
# decoder_a = EdgePredictor(args.embed_dim)
# decoder_d = EdgePredictor(args.embed_dim)
encoder = torch.load('pretrained/encoder.pth')
encoder2 = torch.load('pretrained/encoder2.pth')
decoder_pa = torch.load('pretrained/decoder_pa.pth')
decoder_pp = torch.load('pretrained/decoder_pp.pth')
decoder_pv = torch.load('pretrained/decoder_pv.pth')
decoder_list = [decoder_pa, decoder_pp, decoder_pv]

encoder.to(device)
encoder2.to(device)
for decoder in decoder_list:
    decoder.to(device)

## Training Part

In [6]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

torch.cuda.empty_cache()
# profiler = cProfile.Profile()
# profiler.enable()
train_smote(data, edge_indices, target_edge, encoder, encoder2, decoder_list, train_idx, val_idx, test_idx, 
            args, os_mode = 'edge_sm', train_mode = 'preo')
# profiler.disable()

torch.Size([29166, 2])
Training report
               precision    recall  f1-score   support

           0       0.79      0.43      0.55      4000
           1       0.61      0.88      0.72      4000

    accuracy                           0.66      8000
   macro avg       0.70      0.66      0.64      8000
weighted avg       0.70      0.66      0.64      8000
 
ROC_AUC_SCORE: 0.76630115625

Validation report               precision    recall  f1-score   support

           0       0.79      0.43      0.56      4000
           1       0.61      0.89      0.72      4000

    accuracy                           0.66      8000
   macro avg       0.70      0.66      0.64      8000
weighted avg       0.70      0.66      0.64      8000
 
ROC_AUC_SCORE: 0.7734396875

Testing report
               precision    recall  f1-score   support

           0       0.78      0.43      0.55      6000
           1       0.61      0.88      0.72      6000

    accuracy                           0.65    

(0.55, 0.72, 0.65, 0.63, 0.7673415)

Testing report:

1) NO
               precision    recall  f1-score   support

           0       0.70      0.72      0.71      6000
           1       0.71      0.69      0.70      6000

    accuracy                           0.70     12000
   macro avg       0.70      0.70      0.70     12000
weighted avg       0.70      0.70      0.70     12000

2) UP:

Testing report
               precision    recall  f1-score   support

           0       0.72      0.72      0.72      6000
           1       0.72      0.72      0.72      6000

    accuracy                           0.72     12000
   macro avg       0.72      0.72      0.72     12000
weighted avg       0.72      0.72      0.72     12000

3) SMOTE:

Testing report
               precision    recall  f1-score   support

           0       0.72      0.72      0.72      6000
           1       0.72      0.72      0.72      6000

    accuracy                           0.72     12000
   macro avg       0.72      0.72      0.72     12000
weighted avg       0.72      0.72      0.72     12000

4) Reweight:

Testing report
               precision    recall  f1-score   support

           0       0.73      0.66      0.70      6000
           1       0.69      0.76      0.72      6000

    accuracy                           0.71     12000
   macro avg       0.71      0.71      0.71     12000
weighted avg       0.71      0.71      0.71     12000

5) Embed_sm:

Testing report
               precision    recall  f1-score   support

           0       0.70      0.70      0.70      6000
           1       0.70      0.70      0.70      6000

    accuracy                           0.70     12000
   macro avg       0.70      0.70      0.70     12000
weighted avg       0.70      0.70      0.70     12000

6) gsm:

Testing report
               precision    recall  f1-score   support

           0       0.71      0.71      0.71      6000
           1       0.71      0.71      0.71      6000

    accuracy                           0.71     12000
   macro avg       0.71      0.71      0.71     12000
weighted avg       0.71      0.71      0.71     12000

7) edge_sm:

Testing report
               precision    recall  f1-score   support

           0       0.72      0.72      0.72      6000
           1       0.72      0.72      0.72      6000

    accuracy                           0.72     12000
   macro avg       0.72      0.72      0.72     12000
weighted avg       0.72      0.72      0.72     12000

In [7]:
# profiler.print_stats(sort='cumulative')  # Print results to the console
# # Or save to a file
# with open('profile_results.txt', 'w') as f:
#     profiler.dump_stats(f)

In [8]:
# test_smote(data, edge_indices, data['e']['x'], encoder, classifier, decoder_list, test_idx, 
#            args = args, dataset = 'Test', os_mode = 'no', train_mode = 'noFT')

NO:

|           | Precision  | Recall  | F1-Score  | Support  |
|-----------|------------|---------|-----------|----------|
| Training  |            |         |           |          |
| 0         | 0.67       | 1.00    | 0.80      | 4000     |
| 1         | 0.00       | 0.00    | 0.00      | 2000     |
| Accuracy  |            |         | 0.67      | 6000     |
| Macro Avg | 0.33       | 0.50    | 0.40      | 6000     |
| Weighted Avg | 0.44    | 0.67    | 0.53      | 6000     |

|           | Precision  | Recall  | F1-Score  | Support  |
|-----------|------------|---------|-----------|----------|
| Validation |            |         |           |          |
| 0         | 0.50       | 1.00    | 0.67      | 4000     |
| 1         | 0.78       | 0.00    | 0.00      | 4000     |
| Accuracy  |            |         | 0.50      | 8000     |
| Macro Avg | 0.64       | 0.50    | 0.34      | 8000     |
| Weighted Avg | 0.64    | 0.50    | 0.34      | 8000     |

|           | Precision  | Recall  | F1-Score  | Support  |
|-----------|------------|---------|-----------|----------|
| Testing   |            |         |           |          |
| 0         | 0.50       | 1.00    | 0.67      | 6000     |
| 1         | 0.94       | 0.00    | 0.00      | 6000     |
| Accuracy  |            |         | 0.50      | 12000    |
| Macro Avg | 0.72       | 0.50    | 0.34      | 12000    |
| Weighted Avg | 0.72    | 0.50    | 0.34      | 12000    |


UP:

|           | Precision  | Recall  | F1-Score  | Support  |
|-----------|------------|---------|-----------|----------|
| Training  |            |         |           |          |
| 0         | 0.63       | 0.96    | 0.76      | 4000     |
| 1         | 0.84       | 0.26    | 0.39      | 3000     |
| Accuracy  |            |         | 0.66      | 7000     |
| Macro Avg | 0.74       | 0.61    | 0.58      | 7000     |
| Weighted Avg | 0.72    | 0.66    | 0.61      | 7000     |

|           | Precision  | Recall  | F1-Score  | Support  |
|-----------|------------|---------|-----------|----------|
| Validation |            |         |           |          |
| 0         | 0.56       | 0.95    | 0.71      | 4000     |
| 1         | 0.84       | 0.26    | 0.40      | 4000     |
| Accuracy  |            |         | 0.61      | 8000     |
| Macro Avg | 0.70       | 0.61    | 0.55      | 8000     |
| Weighted Avg | 0.70    | 0.61    | 0.55      | 8000     |

|           | Precision  | Recall  | F1-Score  | Support  |
|-----------|------------|---------|-----------|----------|
| Testing   |            |         |           |          |
| 0         | 0.56       | 0.96    | 0.71      | 6000     |
| 1         | 0.86       | 0.26    | 0.40      | 6000     |
| Accuracy  |            |         | 0.61      | 12000    |
| Macro Avg | 0.71       | 0.61    | 0.55      | 12000    |
| Weighted Avg | 0.71    | 0.61    | 0.55      | 12000    |


3) SMOTE

|           | Precision  | Recall  | F1-Score  | Support  |
|-----------|------------|---------|-----------|----------|
| Training  |            |         |           |          |
| 0         | 0.64       | 0.97    | 0.77      | 4000     |
| 1         | 0.86       | 0.29    | 0.43      | 3000     |
| Accuracy  |            |         | 0.68      | 7000     |
| Macro Avg | 0.75       | 0.63    | 0.60      | 7000     |
| Weighted Avg | 0.74    | 0.68    | 0.63      | 7000     |

|           | Precision  | Recall  | F1-Score  | Support  |
|-----------|------------|---------|-----------|----------|
| Validation |            |         |           |          |
| 0         | 0.56       | 0.95    | 0.70      | 4000     |
| 1         | 0.84       | 0.25    | 0.38      | 4000     |
| Accuracy  |            |         | 0.60      | 8000     |
| Macro Avg | 0.70       | 0.60    | 0.54      | 8000     |
| Weighted Avg | 0.70    | 0.60    | 0.54      | 8000     |

|           | Precision  | Recall  | F1-Score  | Support  |
|-----------|------------|---------|-----------|----------|
| Testing   |            |         |           |          |
| 0         | 0.56       | 0.96    | 0.71      | 6000     |
| 1         | 0.86       | 0.25    | 0.39      | 6000     |
| Accuracy  |            |         | 0.61      | 12000    |
| Macro Avg | 0.71       | 0.61    | 0.55      | 12000    |
| Weighted Avg | 0.71    | 0.61    | 0.55      | 12000    |


4) REWEIGHT

|           | Precision  | Recall  | F1-Score  | Support  |
|-----------|------------|---------|-----------|----------|
| Training  |            |         |           |          |
| 0         | 0.84       | 0.73    | 0.78      | 4000     |
| 1         | 0.57       | 0.72    | 0.64      | 2000     |
| Accuracy  |            |         | 0.73      | 6000     |
| Macro Avg | 0.70       | 0.72    | 0.71      | 6000     |
| Weighted Avg | 0.75    | 0.73    | 0.73      | 6000     |

|           | Precision  | Recall  | F1-Score  | Support  |
|-----------|------------|---------|-----------|----------|
| Validation |            |         |           |          |
| 0         | 0.72       | 0.73    | 0.73      | 4000     |
| 1         | 0.73       | 0.72    | 0.72      | 4000     |
| Accuracy  |            |         | 0.73      | 8000     |
| Macro Avg | 0.73       | 0.73    | 0.73      | 8000     |
| Weighted Avg | 0.73    | 0.73    | 0.73      | 8000     |

|           | Precision  | Recall  | F1-Score  | Support  |
|-----------|------------|---------|-----------|----------|
| Testing   |            |         |           |          |
| 0         | 0.72       | 0.72    | 0.72      | 6000     |
| 1         | 0.72       | 0.72    | 0.72      | 6000     |
| Accuracy  |            |         | 0.72      | 12000    |
| Macro Avg | 0.72       | 0.72    | 0.72      | 12000    |
| Weighted Avg | 0.72    | 0.72    | 0.72      | 12000    |


5) EMBED_SM

|           | Precision  | Recall  | F1-Score  | Support  |
|-----------|------------|---------|-----------|----------|
| Training  |            |         |           |          |
| 0         | 0.69       | 0.99    | 0.81      | 4000     |
| 1         | 0.96       | 0.40    | 0.57      | 3000     |
| Accuracy  |            |         | 0.74      | 7000     |
| Macro Avg | 0.82       | 0.70    | 0.69      | 7000     |
| Weighted Avg | 0.80    | 0.74    | 0.71      | 7000     |

|           | Precision  | Recall  | F1-Score  | Support  |
|-----------|------------|---------|-----------|----------|
| Validation |            |         |           |          |
| 0         | 0.53       | 0.99    | 0.69      | 4000     |
| 1         | 0.90       | 0.12    | 0.22      | 4000     |
| Accuracy  |            |         | 0.55      | 8000     |
| Macro Avg | 0.72       | 0.55    | 0.45      | 8000     |
| Weighted Avg | 0.72    | 0.55    | 0.45      | 8000     |

|           | Precision  | Recall  | F1-Score  | Support  |
|-----------|------------|---------|-----------|----------|
| Testing   |            |         |           |          |
| 0         | 0.53       | 0.99    | 0.69      | 6000     |
| 1         | 0.90       | 0.12    | 0.21      | 6000     |
| Accuracy  |            |         | 0.55      | 12000    |
| Macro Avg | 0.72       | 0.55    | 0.45      | 12000    |
| Weighted Avg | 0.72    | 0.55    | 0.45      | 12000    |


6) GSM

|           | Precision  | Recall  | F1-Score  | Support  |
|-----------|------------|---------|-----------|----------|
| Training  |            |         |           |          |
| 0         | 0.69       | 0.99    | 0.81      | 4000     |
| 1         | 0.96       | 0.40    | 0.57      | 3000     |
| Accuracy  |            |         | 0.74      | 7000     |
| Macro Avg | 0.82       | 0.70    | 0.69      | 7000     |
| Weighted Avg | 0.80    | 0.74    | 0.71      | 7000     |

|           | Precision  | Recall  | F1-Score  | Support  |
|-----------|------------|---------|-----------|----------|
| Validation |            |         |           |          |
| 0         | 0.53       | 0.99    | 0.69      | 4000     |
| 1         | 0.90       | 0.12    | 0.22      | 4000     |
| Accuracy  |            |         | 0.55      | 8000     |
| Macro Avg | 0.72       | 0.55    | 0.45      | 8000     |
| Weighted Avg | 0.72    | 0.55    | 0.45      | 8000     |

|           | Precision  | Recall  | F1-Score  | Support  |
|-----------|------------|---------|-----------|----------|
| Testing   |            |         |           |          |
| 0         | 0.53       | 0.99    | 0.69      | 6000     |
| 1         | 0.90       | 0.12    | 0.21      | 6000     |
| Accuracy  |            |         | 0.55      | 12000    |
| Macro Avg | 0.72       | 0.55    | 0.45      | 12000    |
| Weighted Avg | 0.72    | 0.55    | 0.45      | 12000    |


7) EDGE SM

|           | Precision  | Recall  | F1-Score  | Support  |
|-----------|------------|---------|-----------|----------|
| Training  |            |         |           |          |
| 0         | 0.69       | 0.99    | 0.81      | 4000     |
| 1         | 0.96       | 0.40    | 0.57      | 3000     |
| Accuracy  |            |         | 0.74      | 7000     |
| Macro Avg | 0.82       | 0.70    | 0.69      | 7000     |
| Weighted Avg | 0.80    | 0.74    | 0.71      | 7000     |

|           | Precision  | Recall  | F1-Score  | Support  |
|-----------|------------|---------|-----------|----------|
| Validation |            |         |           |          |
| 0         | 0.53       | 0.99    | 0.69      | 4000     |
| 1         | 0.90       | 0.12    | 0.22      | 4000     |
| Accuracy  |            |         | 0.55      | 8000     |
| Macro Avg | 0.72       | 0.55    | 0.45      | 8000     |
| Weighted Avg | 0.72    | 0.55    | 0.45      | 8000     |

|           | Precision  | Recall  | F1-Score  | Support  |
|-----------|------------|---------|-----------|----------|
| Testing   |            |         |           |          |
| 0         | 0.53       | 0.99    | 0.69      | 6000     |
| 1         | 0.90       | 0.12    | 0.21      | 6000     |
| Accuracy  |            |         | 0.55      | 12000    |
| Macro Avg | 0.72       | 0.55    | 0.45      | 12000    |
| Weighted Avg | 0.72    | 0.55    | 0.45      | 12000    |
